In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10, MNIST
from torchvision.transforms import Compose, ToTensor, Normalize, RandomHorizontalFlip, RandomRotation

from cr_ibp_modules import *
from cr_ibp_utils import *

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm import trange
from copy import deepcopy
    
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
torch.manual_seed(42);

In [49]:
dataset = CIFAR10('../data/cifar10',
                  download=True,
                  train=True,
                  transform=Compose([ToTensor(), Normalize(means, stds)]))
BATCH_SIZE = 32
IMAGE_SIZE = dataset[0][0].shape[1:]
FC_DIM = np.prod(IMAGE_SIZE)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [40000, 10000])
dataloader_train = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dataloader_val = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
dataloader_test = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
dataloader_vis = DataLoader(dataset, batch_size=1, shuffle=False)

Files already downloaded and verified


In [45]:
means = dataset.train_data.transpose(3, 0, 1, 2).reshape(3, -1).mean(axis=-1)
stds = dataset.train_data.transpose(3, 0, 1, 2).reshape(3, -1).std(axis=-1)

In [50]:
model = Net(n_channels=3, fc_dim=FC_DIM)
opt = torch.optim.Adam(model.parameters())
lossf = nn.CrossEntropyLoss()

In [51]:
train(model, dataloader_train, dataloader_val, lossf, opt,
      lr_schedule=[1e-3] * 10 + [1e-4] * 10 + [1e-5] * 10, epochs=30)

/Users/nickstulov/Desktop/Study/dl/Project/cr_ibp_utils.py:57: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = np.argmax(F.softmax(logits).detach().numpy(), axis=-1)


[ 1]	train loss	2.3032123	time	39s	train acc	0.10


/Users/nickstulov/Desktop/Study/dl/Project/cr_ibp_utils.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = np.argmax(F.softmax(logits).detach().numpy(), axis=-1)


[ 1]	valid loss	2.3028925	valid acc	0.10
[ 2]	train loss	2.3027424	time	41s	train acc	0.10
[ 3]	train loss	2.3027627	time	41s	train acc	0.10
[ 3]	valid loss	2.3029000	valid acc	0.09
[ 4]	train loss	2.3027593	time	39s	train acc	0.10
[ 5]	train loss	2.3027214	time	40s	train acc	0.10
[ 5]	valid loss	2.3028015	valid acc	0.10
[ 6]	train loss	2.3027516	time	37s	train acc	0.10
[ 7]	train loss	2.3027626	time	37s	train acc	0.10
[ 7]	valid loss	2.3028761	valid acc	0.10
[ 8]	train loss	2.3027573	time	38s	train acc	0.10
[ 9]	train loss	2.3027552	time	36s	train acc	0.10
[ 9]	valid loss	2.3029673	valid acc	0.10
[10]	train loss	2.3027663	time	42s	train acc	0.10
[11]	train loss	2.3026081	time	41s	train acc	0.10
[11]	valid loss	2.3029347	valid acc	0.10
[12]	train loss	2.3025979	time	35s	train acc	0.10
[13]	train loss	2.3025892	time	36s	train acc	0.10
[13]	valid loss	2.3029142	valid acc	0.09
[14]	train loss	2.3025844	time	39s	train acc	0.10
[15]	train loss	2.3025822	time	39s	train acc	0.10
[15]	valid lo

KeyboardInterrupt: 

In [ ]:
torch.save(model, '../models/model_clean.mdl')

In [ ]:
model_ibp = Net(n_channels=3, fc_dim=FC_DIM)
opt = torch.optim.Adam(model_ibp.parameters())
lossf = nn.CrossEntropyLoss()

In [ ]:
train(model_ibp, dataloader_train, dataloader_val, lossf, opt,
      lr_schedule=[1e-3] * 15 + [1e-4] * 10 + [1e-5] * 5, mode='ibp', epochs=30)